In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Create Table

In [2]:
# import library for request web page
from urllib.request import urlopen
optionsUrl = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
optionsPage = urlopen(optionsUrl)

In [3]:
# import BS library for pulling data out of HTML (wikipedia). load Wikipedia and parse it with Beautiful Soup.
from bs4 import BeautifulSoup
soup = BeautifulSoup(optionsPage)

In [4]:
# Extract all the <table> tags
My_table = soup.find('table',{'class':'wikitable sortable'})

In [5]:
# search for the one with the headings corresponding to the data we want. Then iterate over its rows, pulling out the columns we want and writing the cell text to the file 

table_rows = My_table.find_all('tr')
res = [] 

for tr in table_rows:
        tds = tr.find_all('td')
        row = [td.text.strip() for td in tds if tr.text.strip()]
        if row:
            res.append(row)

# transform the data in the table on the Wikipedia page into pandas dataframe
df = pd.DataFrame(res, columns=["Postalcode", "Borough", "Neighborhood"])
df.head(20)

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [6]:
df.shape

(289, 3)

In [7]:
#Ignore cells with a borough that is Not assigned.

df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.reset_index(drop = True, inplace = True)
df.head(15)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [8]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

df.loc[df['Neighborhood']=="Not assigned",'Neighborhood']=df.loc[df['Neighborhood']=="Not assigned",'Borough']
df.reset_index(drop = True, inplace = True)
df.head(20)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [9]:
# More than one neighborhood can exist in one postal code area. Such rows will be combined into one row with the neighborhoods separated with a comma 

df.Neighborhood = df.Neighborhood.astype(str)
toronto_df = df.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
toronto_df.head(20)

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [10]:
# use the .shape method to print the number of rows of your dataframe
toronto_df.shape

(103, 3)

In [11]:
geo_df=pd.read_csv("https://cocl.us/Geospatial_data")
geo_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [12]:
#put location data in data frame
toronto_df['Latitude'] = geo_df['Latitude'].values
toronto_df['Longitude'] = geo_df['Longitude'].values
toronto_df

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Explore and cluster the neighborhoods in Toronto.

In [13]:
# for illustration segment and cluster only the neighborhoods in North York. We slice the original dataframe and create a new dataframe of North York data.
ny_data = toronto_df[toronto_df['Borough'] == 'North York'].reset_index(drop=True)
ny_data.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M2H,North York,Hillcrest Village,43.803762,-79.363452
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
2,M2K,North York,Bayview Village,43.786947,-79.385975
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493


In [15]:
address = 'North York, Toronto'

geolocator = Nominatim(user_agent="northyork_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7708175, -79.4132998.


In [17]:
# create map of North York using latitude and longitude values
map_ny = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(ny_data['Latitude'], ny_data['Longitude'], ny_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ny)  
    
map_ny

In [19]:
# Define Foursquare Credentials and Version

CLIENT_ID = '' # Foursquare ID
CLIENT_SECRET = '' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

# Explore Neighborhoods in North York.


### Create a function to get latitude, longitude, request url and find venues to all the neighborhoods in North York

In [38]:
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [91]:
def getNearbyVenues(names, latitudes, longitudes, radius=1500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url_ny = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results_ny = requests.get(url_ny).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results_ny])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [92]:
 # write the code to run the above function on each neighborhood and create a new dataframe called ny_venues.
ny_venues = getNearbyVenues(names=ny_data['Neighborhood'],
                                   latitudes=ny_data['Latitude'],
                                   longitudes=ny_data['Longitude']
                                  )

Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Maple Leaf Park, North Park, Upwood Park
Humber Summit
Emery, Humberlea


In [93]:
ny_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest Village,43.803762,-79.363452,Bayview Golf & Country Club,43.809391,-79.375285,Golf Course
1,Hillcrest Village,43.803762,-79.363452,Tastee,43.807722,-79.356798,Bakery
2,Hillcrest Village,43.803762,-79.363452,고려삼계탕 Korean Ginseng Chicken Soup & Bibimbap,43.798391,-79.369187,Korean Restaurant
3,Hillcrest Village,43.803762,-79.363452,Chatime Willowdale,43.791326,-79.367506,Bubble Tea Shop
4,Hillcrest Village,43.803762,-79.363452,Galati,43.797831,-79.369410,Grocery Store


In [94]:
# Let's check how many venues were returned for each neighborhood
ny_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",36,36,36,36,36,36
Bayview Village,16,16,16,16,16,16
"Bedford Park, Lawrence Manor East",77,77,77,77,77,77
"CFB Toronto, Downsview East",26,26,26,26,26,26
Don Mills North,89,89,89,89,89,89
Downsview Central,30,30,30,30,30,30
Downsview Northwest,27,27,27,27,27,27
Downsview West,10,10,10,10,10,10
"Emery, Humberlea",15,15,15,15,15,15


In [95]:
# find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(ny_venues['Venue Category'].unique())))

There are 186 uniques categories.


### Analyze Each Neighborhood

In [96]:
# one hot encoding
ny_onehot = pd.get_dummies(ny_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
ny_onehot['Neighborhood'] = ny_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [ny_onehot.columns[-1]] + list(ny_onehot.columns[:-1])
ny_onehot = ny_onehot[fixed_columns]

ny_onehot.head(15)

,Neighborhood,Accessories Store,Airport,American Restaurant,Argentinian Restaurant,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,BBQ Joint,...,Turkish Restaurant,Udon Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [97]:
# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
ny_grouped = ny_onehot.groupby('Neighborhood').mean().reset_index()
ny_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Argentinian Restaurant,Arts & Crafts Store,Asian Restaurant,Astrologer,Athletics & Sports,BBQ Joint,...,Turkish Restaurant,Udon Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wings Joint,Women's Store,Yoga Studio
0,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.000000,0.027778,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.012987,0.000000,0.000000,0.038961,0.00,0.000000,0.012987,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.012987,0.000000,0.000000
3,"CFB Toronto, Downsview East",0.000000,0.038462,0.000000,0.000000,0.038462,0.000000,0.00,0.076923,0.000000,...,0.076923,0.00,0.000000,0.000000,0.038462,0.000000,0.038462,0.000000,0.000000,0.000000
4,Don Mills North,0.000000,0.000000,0.022472,0.000000,0.000000,0.022472,0.00,0.000000,0.000000,...,0.000000,0.00,0.011236,0.000000,0.000000,0.000000,0.000000,0.011236,0.011236,0.000000
5,Downsview Central,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Downsview Northwest,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.037037,0.000000,0.000000,0.000000,0.000000,0.000000
7,Downsview West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Emery, Humberlea",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Fairview, Henry Farm, Oriole",0.000000,0.000000,0.016667,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,...,0.000000,0.00,0.016667,0.000000,0.000000,0.000000,0.000000,0.016667,0.000000,0.000000


In [98]:
# print each neighborhood along with the top 5 most common venues
num_top_venues = 5

for hood in ny_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = ny_grouped[ny_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
                 venue  freq
0                 Park  0.14
1          Pizza Place  0.08
2          Coffee Shop  0.08
3        Deli / Bodega  0.03
4  Fried Chicken Joint  0.03


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.12
1                 Bank  0.12
2                Trail  0.12
3   Chinese Restaurant  0.06
4        Grocery Store  0.06


----Bedford Park, Lawrence Manor East----
              venue  freq
0            Bakery  0.09
1       Coffee Shop  0.08
2  Sushi Restaurant  0.05
3        Bagel Shop  0.05
4              Café  0.04


----CFB Toronto, Downsview East----
                  venue  freq
0  Gym / Fitness Center  0.12
1    Turkish Restaurant  0.08
2             Racetrack  0.08
3    Athletics & Sports  0.08
4           Coffee Shop  0.08


----Don Mills North----
                 venue  freq
0          Coffee Shop  0.08
1  Japanese Restaurant  0.08
2           Restaurant  0.04
3         

In [99]:
# put that into a pandas dataframe

# create the new dataframe and display the top 10 venues for each neighborhood.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
ny_venues_sorted = pd.DataFrame(columns=columns)
ny_venues_sorted['Neighborhood'] = ny_grouped['Neighborhood']

for ind in np.arange(ny_grouped.shape[0]):
    ny_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ny_grouped.iloc[ind, :], num_top_venues)

ny_venues_sorted.head(10)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Park,Pizza Place,Coffee Shop,Bridal Shop,Shopping Mall,Sushi Restaurant,French Restaurant,Building,Ski Chalet,Fast Food Restaurant
1,Bayview Village,Bank,Japanese Restaurant,Trail,Flower Shop,Convenience Store,Skating Rink,Shopping Mall,Café,Park,Fast Food Restaurant
2,"Bedford Park, Lawrence Manor East",Bakery,Coffee Shop,Sushi Restaurant,Bagel Shop,Café,Italian Restaurant,Asian Restaurant,Restaurant,Pub,Pizza Place
3,"CFB Toronto, Downsview East",Gym / Fitness Center,Coffee Shop,Turkish Restaurant,Racetrack,Athletics & Sports,Climbing Gym,Basketball Court,Gym,Jewelry Store,Liquor Store
4,Don Mills North,Japanese Restaurant,Coffee Shop,Restaurant,Pizza Place,Bank,Park,Burger Joint,Ice Cream Shop,Bar,Middle Eastern Restaurant
5,Downsview Central,Vietnamese Restaurant,Pharmacy,Coffee Shop,Park,Supermarket,Beer Store,Restaurant,Sandwich Place,New American Restaurant,Burrito Place
6,Downsview Northwest,Hotel,Tennis Stadium,Pizza Place,Pharmacy,Sandwich Place,Fast Food Restaurant,Coffee Shop,Theater,Historic Site,Skating Rink
7,Downsview West,Park,Coffee Shop,Plaza,Vietnamese Restaurant,Pizza Place,Moving Target,Tea Room,Bank,Drugstore,Fish & Chips Shop
8,"Emery, Humberlea",Coffee Shop,Golf Course,Sandwich Place,Japanese Restaurant,Electronics Store,Furniture / Home Store,Park,Beer Store,Liquor Store,Nightclub
9,"Fairview, Henry Farm, Oriole",Coffee Shop,Clothing Store,Park,Middle Eastern Restaurant,Japanese Restaurant,Pharmacy,Pizza Place,Sandwich Place,Bakery,Electronics Store


## Cluster Neighborhoods

In [113]:
# set number of clusters
kclusters = 5

ny_grouped_clustering = ny_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(ny_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:15] 

array([0, 4, 0, 2, 0, 3, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [110]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood
# add clustering labels

ny_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)



ValueError: cannot insert Cluster Labels, already exists

In [114]:
ny_merged = ny_data

# merge ny_grouped with ny_data to add latitude/longitude for each neighborhood
ny_merged = ny_merged.join(ny_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ny_merged.head(23) # check the last columns!

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M2H,North York,Hillcrest Village,43.803762,-79.363452,0,Coffee Shop,Chinese Restaurant,Park,Pharmacy,Bank,Sushi Restaurant,Pizza Place,Grocery Store,Sandwich Place,Bakery
1,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0,Coffee Shop,Clothing Store,Park,Middle Eastern Restaurant,Japanese Restaurant,Pharmacy,Pizza Place,Sandwich Place,Bakery,Electronics Store
2,M2K,North York,Bayview Village,43.786947,-79.385975,4,Bank,Japanese Restaurant,Trail,Flower Shop,Convenience Store,Skating Rink,Shopping Mall,Café,Park,Fast Food Restaurant
3,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,0,Coffee Shop,Bank,Furniture / Home Store,Japanese Restaurant,Supermarket,Pharmacy,Fast Food Restaurant,Park,Burger Joint,Pizza Place
4,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,0,Korean Restaurant,Coffee Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Shopping Mall,Juice Bar,Fast Food Restaurant,Bank
5,M2N,North York,Willowdale South,43.770120,-79.408493,0,Korean Restaurant,Coffee Shop,Bubble Tea Shop,Japanese Restaurant,Café,Grocery Store,Ramen Restaurant,Pizza Place,Fried Chicken Joint,Sushi Restaurant
6,M2P,North York,York Mills West,43.752758,-79.400049,0,Coffee Shop,Bank,Park,Pharmacy,Fast Food Restaurant,Japanese Restaurant,Fried Chicken Joint,Gym,Sandwich Place,Burger Joint
7,M2R,North York,Willowdale West,43.782736,-79.442259,0,Coffee Shop,Pizza Place,Pharmacy,Park,Sandwich Place,Bagel Shop,Shopping Mall,Skating Rink,Fast Food Restaurant,Business Service
8,M3A,North York,Parkwoods,43.753259,-79.329656,0,Pharmacy,Coffee Shop,Bus Stop,Park,Bank,Health Food Store,Sandwich Place,Fast Food Restaurant,Skating Rink,Bus Line
9,M3B,North York,Don Mills North,43.745906,-79.352188,0,Japanese Restaurant,Coffee Shop,Restaurant,Pizza Place,Bank,Park,Burger Joint,Ice Cream Shop,Bar,Middle Eastern Restaurant


In [115]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ny_merged['Latitude'], ny_merged['Longitude'], ny_merged['Neighborhood'], ny_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

## Cluster 1

In [104]:
ny_merged.loc[ny_merged['Cluster Labels'] == 0, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,0,Coffee Shop,Chinese Restaurant,Park,Pharmacy,Bank,Sushi Restaurant,Pizza Place,Grocery Store,Sandwich Place,Bakery
1,North York,0,Coffee Shop,Clothing Store,Park,Middle Eastern Restaurant,Japanese Restaurant,Pharmacy,Pizza Place,Sandwich Place,Bakery,Electronics Store
3,North York,0,Coffee Shop,Bank,Furniture / Home Store,Japanese Restaurant,Supermarket,Pharmacy,Fast Food Restaurant,Park,Burger Joint,Pizza Place
4,North York,0,Korean Restaurant,Coffee Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Shopping Mall,Juice Bar,Fast Food Restaurant,Bank
5,North York,0,Korean Restaurant,Coffee Shop,Bubble Tea Shop,Japanese Restaurant,Café,Grocery Store,Ramen Restaurant,Pizza Place,Fried Chicken Joint,Sushi Restaurant
6,North York,0,Coffee Shop,Bank,Park,Pharmacy,Fast Food Restaurant,Japanese Restaurant,Fried Chicken Joint,Gym,Sandwich Place,Burger Joint
7,North York,0,Coffee Shop,Pizza Place,Pharmacy,Park,Sandwich Place,Bagel Shop,Shopping Mall,Skating Rink,Fast Food Restaurant,Business Service
8,North York,0,Pharmacy,Coffee Shop,Bus Stop,Park,Bank,Health Food Store,Sandwich Place,Fast Food Restaurant,Skating Rink,Bus Line
9,North York,0,Japanese Restaurant,Coffee Shop,Restaurant,Pizza Place,Bank,Park,Burger Joint,Ice Cream Shop,Bar,Middle Eastern Restaurant
10,North York,0,Coffee Shop,Gym,Japanese Restaurant,Middle Eastern Restaurant,Restaurant,Sandwich Place,Asian Restaurant,Chinese Restaurant,Park,Movie Theater


## Cluster 2

In [105]:
ny_merged.loc[ny_merged['Cluster Labels'] == 1, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,North York,1,Park,Coffee Shop,Plaza,Vietnamese Restaurant,Pizza Place,Moving Target,Tea Room,Bank,Drugstore,Fish & Chips Shop


## Cluster 3

In [106]:
ny_merged.loc[ny_merged['Cluster Labels'] == 2, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,North York,2,Gym / Fitness Center,Coffee Shop,Turkish Restaurant,Racetrack,Athletics & Sports,Climbing Gym,Basketball Court,Gym,Jewelry Store,Liquor Store


## Cluster 4

In [107]:
ny_merged.loc[ny_merged['Cluster Labels'] == 3, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,North York,3,Vietnamese Restaurant,Pharmacy,Coffee Shop,Park,Supermarket,Beer Store,Restaurant,Sandwich Place,New American Restaurant,Burrito Place


## Cluster 5

In [108]:
ny_merged.loc[ny_merged['Cluster Labels'] == 4, ny_merged.columns[[1] + list(range(5, ny_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,North York,4,Bank,Japanese Restaurant,Trail,Flower Shop,Convenience Store,Skating Rink,Shopping Mall,Café,Park,Fast Food Restaurant
